In [19]:
import time
import os
import numpy as np
import pandas as pd
from module import in_fun
from matplotlib import pyplot as plt
from module.torch import BinaryClassifier
from datetime import datetime

%matplotlib widget

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [21]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

print(device)

cpu


In [22]:
net_abs = BinaryClassifier(input_size = 9)
net_diff = BinaryClassifier(input_size = 9)
net_each = BinaryClassifier(input_size = 18)
net_abs.load_state_dict(torch.load("./model/abs"))
net_diff.load_state_dict(torch.load("./model/diff"))
net_each.load_state_dict(torch.load("./model/each"))

<All keys matched successfully>

In [23]:
data_path = "data/app_test/06/"
file_list = os.listdir(data_path)
file_list_csv = [file for file in file_list if file.endswith(".csv")]
print(file_list_csv)

['1.csv', '2.csv']


In [24]:
data_list = []
for file_csv in file_list_csv:
    data_list.append(pd.read_csv(
            data_path + file_csv,
            header=None,
        ).to_numpy())

In [25]:
plt.figure()
for idx, data in enumerate(data_list):
    plt.scatter(pd.to_datetime(data[:,0]/1000, unit='s'), np.ones_like(data[:,0])*(idx+1), label = file_list_csv[idx], s = 5)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
plt.figure()
data=data_list[0]
plt.scatter(pd.to_datetime(data[:,0]/1000, unit='s'), data[:,24], s = 5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# for idx, data in enumerate(data_list):
    print(f"{file_list_csv[idx][:-4]} start time : {datetime.fromtimestamp(data[0,0]/1000)}")

In [9]:
start_time = datetime(2021, 3, 26, hour=6, minute=5, second=0)
end_time = datetime(2021, 3, 26, hour=6, minute=46, second=0)
start_timestamp = time.mktime(start_time.timetuple())
end_timestamp = time.mktime(end_time.timetuple())
print(start_time)
print(start_timestamp)
print(end_time)
print(end_timestamp)
for idx in np.arange(len(data_list)):
    print(np.shape(data_list[idx]))
    data_list[idx] = data_list[idx][data_list[idx][:,0] >= start_timestamp * 1000]
    data_list[idx] = data_list[idx][data_list[idx][:,0] <= end_timestamp * 1000]
    
    print(np.shape(data_list[idx]))

2021-03-26 06:05:00
1616738700.0
2021-03-26 06:46:00
1616741160.0
(30798, 51)
(0, 51)
(22120, 51)
(0, 51)
(22619, 51)
(0, 51)


In [10]:
plt.figure()
for idx, data in enumerate(data_list):
    plt.scatter(pd.to_datetime(data[:,0]/1000, unit='s'), np.ones_like(data[:,0])*(idx+1), label = file_list_csv[idx], s=5)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
parsed_list = []
beacon_list = [1, 2, 3, 4, 10, 6, 7, 8, 9]
for data in data_list:
    parsed_list.append(in_fun.parser(data, beacon_list, beacon_length=7))
    print(np.shape(parsed_list[-1]))    

(0, 32)
(0, 32)
(0, 32)


In [12]:
plt.figure()
temp = parsed_list[0][:, -9:]
print(np.shape(temp))
plt.plot(temp[:,7], label=f"beacon {idx+1}")
plt.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0, 9)


In [13]:
cur_timestamp = int(start_timestamp)
temp_list = [[],[],[],[],[],[],[]]
while cur_timestamp < end_timestamp:
    cur_list = []
    confirmed=np.zeros([0,9])
    for idx, parsed in enumerate(parsed_list):
        parsed = parsed[parsed[:, 0] >= cur_timestamp * 1000]
        parsed = parsed[parsed[:, 0] <= (cur_timestamp + 30) * 1000]
        if len(parsed) is 0:
            continue
        if idx is 0:
            confirmed = np.max(parsed[:, -9:], axis = 0).reshape([1, 9])
            confirmed += 200
            confirmed = confirmed / 200
        elif np.shape(confirmed)[0] is not 0 and np.sum(confirmed) is not 0:
            contact = np.max(parsed[:, -9:], axis = 0).reshape([1, 9])
            contact += 200
            contact = contact / 200
            if np.shape(contact)[0] is not  0:
                res_abs, res_diff, res_each = in_fun.matcher(confirmed, contact)
                result_abs = net_abs(torch.from_numpy(res_abs).float())
                if result_abs >= 0.5:
#                     print(f"{file_list_csv[idx][:-4]} confirmed at {datetime.fromtimestamp(cur_timestamp)}")
                    pass
                result_diff = net_diff(torch.from_numpy(res_diff).float())
                if result_diff >= 0.5:
#                     print(res_diff)
#                     print(f"{file_list_csv[idx][:-4]} contacted at {datetime.fromtimestamp(cur_timestamp)}")
                    pass
            
                result_each = net_each(torch.from_numpy(res_each).float())
                if result_each >= 0.5:
                    print(res_each)
                    print(f"{file_list_csv[idx][:-4]} contacted at {datetime.fromtimestamp(cur_timestamp)}")
                    print(cur_timestamp)
                    temp_list[idx-1].append(datetime.fromtimestamp(cur_timestamp))
                    pass
    
                    
    cur_timestamp += 30